# OpenSearch + DeepSeek 实现 RAG 知识库

# 传统的 RAG 方案

![image-tradition](RAG-Tradition.png)

<br>
<br>
<br>
<br>
<br>

# OpenSearch 实现 RAG 知识库方案架构

![arch](RAG-OpenSearch.png)

# 操作步骤

1. 环境准备
2. 设置相关权限（IAM Role/IAM Policy/OpenSearch Security）
3. 在 OpenSearch 创建 ML Connector(DeepSeek Connector/Embedding Connector)
4. 在 OpenSearch 创建 Model(DeepSeek Model/Embedding Model),从 OpenSearch 测试 DeepSeek Model
5. 导入知识库
6. 在 OpenSearch 创建 Search Pipeline
7. 测试问答

# 1. 环境准备

## 1.1 前提条件

* 预先在 Sagemaker 中部署好两个模型：1. DeepSeek, 2. BGE Embedding
* 创建 OpenSearch 集群

## 1.2 初始化

In [2]:
# 安装依赖
!pip install opensearch-py
!pip install requests-aws4auth

  Using cached opensearch_py-2.8.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached Events-0.5-py3-none-any.whl.metadata (3.9 kB)
Using cached opensearch_py-2.8.0-py3-none-any.whl (353 kB)
Using cached Events-0.5-py3-none-any.whl (6.8 kB)
  Using cached requests_aws4auth-1.3.1-py3-none-any.whl.metadata (18 kB)
Using cached requests_aws4auth-1.3.1-py3-none-any.whl (24 kB)


In [9]:
import boto3
import json
from botocore.exceptions import ClientError

def get_opensearch_credentials(secret_name, region_name="us-east-1"):
    """
    从AWS Secret Manager获取OpenSearch的凭证
    
    参数:
        secret_name (str): Secret Manager中存储凭证的密钥名称
        region_name (str): AWS区域名称，默认为us-east-1
        
    返回:
        dict: 包含用户名和密码的字典
    """
    # 创建Secrets Manager客户端
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )
    
    try:
        # 获取密钥
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        # 处理可能的错误
        if e.response['Error']['Code'] == 'DecryptionFailureException':
            raise Exception("无法解密密钥值")
        elif e.response['Error']['Code'] == 'InternalServiceErrorException':
            raise Exception("内部服务错误")
        elif e.response['Error']['Code'] == 'InvalidParameterException':
            raise Exception("参数无效")
        elif e.response['Error']['Code'] == 'InvalidRequestException':
            raise Exception("请求无效")
        elif e.response['Error']['Code'] == 'ResourceNotFoundException':
            raise Exception(f"找不到指定的密钥: {secret_name}")
        else:
            raise e
    else:
        # 密钥可能是字符串或二进制，需要处理两种情况
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
            return json.loads(secret)
        else:
            # 如果是二进制，需要解码
            decoded_binary_secret = base64.b64decode(get_secret_value_response['SecretBinary'])
            return json.loads(decoded_binary_secret)

## 1.3 设置环境变量

In [13]:
# 设置环境变量
import os

sts = boto3.client('sts')
account_id = sts.get_caller_identity()['Account']

os.environ["ACCOUNT_ID"] = account_id


# 部署的 Region，us-east-1
os.environ["DEEPSEEK_AWS_REGION"] = "us-east-1"

# OpenSearch 的访问配置
os.environ["OPENSEARCH_SERVICE_DOMAIN_ENDPOINT"] = "https://search-public-vector-4xl-7tpwyy5762g6qmi3xukiasfmhq.us-east-1.es.amazonaws.com"
os.environ["OPENSEARCH_SERVICE_ADMIN_USER"] = "admin"
os.environ["OPENSEARCH_SERVICE_ADMIN_PASSWORD"] = get_opensearch_credentials("opensearch_credentials", os.environ["DEEPSEEK_AWS_REGION"]).get("password")

# 模型相关
os.environ["OPENSEARCH_SERVICE_DOMAIN_ARN"] = f"arn:aws:es:{os.environ['DEEPSEEK_AWS_REGION']}:{os.environ['ACCOUNT_ID']}:domain/deepseek-demo"

print(f"ACCOUNT_ID:{account_id}")
opensearch_arn = os.environ["OPENSEARCH_SERVICE_DOMAIN_ARN"]
print(f"OPENSEARCH_SERVICE_DOMAIN_ARN:{opensearch_arn}")

ACCOUNT_ID:812046859005
OPENSEARCH_SERVICE_DOMAIN_ARN:arn:aws:es:us-east-1:812046859005:domain/deepseek-demo


# 2.设置相关权限（IAM Role/IAM Policy/OpenSearch Security）

## 2.1 创建 IAM 角色 invoke_deepseek_role

创建角色 invoke_deepseek_role，用于 OpenSearch Connector 远程调用 Sagemaker endpoint

In [12]:
import boto3
import json
import os


# The script will create a role and policy with the names below. It
# reads the ARN for the SageMaker endpoint from the environment.
invoke_deepseek_policy_name = 'my_invoke_bedrock_deepseek_model_policy'
invoke_deepseek_role_name = 'my_invoke_bedrock_deepseek_model_role'
bedrock_deepseek_arn = ''
bedrock_titain_embedding_arn = ''


# Allows invoke endpoint
policy = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "bedrock:InvokeModel"
      ],
      "Resource": [
          "arn:aws:bedrock:us-east-1:812046859005:inference-profile/us.deepseek.r1-v1:0",
          "arn:aws:bedrock:us-east-1::foundation-model/deepseek.r1-v1:0"
      ]
    }
  ]
}


# Allows OpenSearch Service to assume the role. The role and policy
# together allow OpenSearch Service to call SageMaker to invoke
# DeepSeek to generate text.
trust_relationship = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "es.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}


# Check for an existing policy with the same name, and error out
# if it exists.
iam = boto3.client('iam')
sts = boto3.client('sts')
existing_policy = None
try:
  # This constructs an ARN for the policy based on the current
  # account. If you need to run this for another account, you can 
  # change the account ID below.
  account_id = sts.get_caller_identity()['Account']
  policy_arn = f'arn:aws:iam::{account_id}:policy/{invoke_deepseek_policy_name}'
  existing_policy = iam.get_policy(PolicyArn=policy_arn)['Policy']
  if existing_policy:
    raise Exception(f"Policy {invoke_deepseek_policy_name} already exists. Please set another policy name")
except iam.exceptions.NoSuchEntityException:
  pass


# Check for an existing policy with the same name, and error out
# if it exists.
existing_role = None
try:
  existing_role = iam.get_role(RoleName=invoke_deepseek_role_name)
  if existing_role:
    raise Exception(f"Role {invoke_deepseek_role_name} already exists. Please set another role name")
except iam.exceptions.NoSuchEntityException:
  pass


# Create the policy
policy = iam.create_policy(
  PolicyName=invoke_deepseek_policy_name,
  PolicyDocument=json.dumps(policy)
)
policy_arn = policy['Policy']['Arn']


# Create the role, with the policy document just created.
role = iam.create_role(
  RoleName=invoke_deepseek_role_name,
  AssumeRolePolicyDocument=json.dumps(trust_relationship)
)
role_arn = role['Role']['Arn']
iam.attach_role_policy(
  RoleName=invoke_deepseek_role_name,
  PolicyArn=policy_arn
)

print(f'Created policy {policy_arn}')
print(f'Created role {role_arn}')

print(f'\nPlease execute the following command\nos.environ["INVOKE_DEEPSEEK_ROLE"] = "{role_arn}"\n')

Created policy arn:aws:iam::812046859005:policy/my_invoke_bedrock_deepseek_model_policy
Created role arn:aws:iam::812046859005:role/my_invoke_bedrock_deepseek_model_role

Please execute the following command
os.environ["INVOKE_DEEPSEEK_ROLE"] = "arn:aws:iam::812046859005:role/my_invoke_bedrock_deepseek_model_role"



## 2.2 创建 IAM 角色 create_deepseek_connector_role

创建一个用于操作 OpenSearch 的角色，后续将使用该角色向 OpenSearch 发送创建/删除/导入数据等操作

In [18]:
# Copyright 2025 Norris
# MIT-0
#
'''
This module constructs an IAM role and policy that enables your account
to create an OpenSearch connector in your Amazon OpenSearch Service domain.
'''

import boto3
import json
import os


# This script will create a role and policy document with 
# the following names.
create_connector_policy_name = 'create_deepseek_connector_policy'
create_connector_role_name = 'create_deepseek_connector_role'

# Read environment variables for the invoke role, and the domain ARNs.
invoke_connector_role_arn = 'arn:aws:iam::812046859005:role/my_invoke_bedrock_deepseek_model_role'
opensearch_service_domain_arn = 'arn:aws:es:us-east-1:812046859005:domain/public-vector-4xl'

def get_current_role_arn():
    sts_client = boto3.client('sts')
    try:
        # 获取当前身份信息
        response = sts_client.get_caller_identity()
        
        # 返回的 ARN 会是角色的 ARN，如果代码在使用角色的环境中运行
        arn = response['Arn']
        
        # 如果需要提取角色名称
        if ':assumed-role/' in arn:
            # 从 ARN 中提取角色名称
            role_name = arn.split(':assumed-role/')[1].split('/')[0]
            # 构建标准的 IAM 角色 ARN
            account_id = response['Account']
            role_arn = f"arn:aws:iam::{account_id}:role/service-role/{role_name}"
            return role_arn
        
        return arn
    except Exception as e:
        print(f"获取当前角色 ARN 时出错: {e}")
        return None

# 使用示例
current_role_arn = get_current_role_arn()


# This policy will allow post operations on the OpenSearch Service
# domain. It adds a pass role so that OpenSearch can validate the
# connector.
policy = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": "iam:PassRole",
      "Resource": invoke_connector_role_arn
    },
    {
      "Effect": "Allow",
      "Action": "es:ESHttpPost",
      "Resource": opensearch_service_domain_arn
    }
  ]
}


# Pulls the current user ARN from Boto's entity resolution, based
# on either aws configure, or environment variables. This role,
# with the policy above enables you to call OpenSearch's
# create_connector API
# current_user_arn = boto3.resource('iam').CurrentUser().arn
trust_relationship = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "AWS": 'arn:aws:iam::812046859005:role/datazone_usr_role_4zdc6o5ho34rl3_6lftaxwm3p3cfb'
      },
      "Action": "sts:AssumeRole"
    }
  ]
}


iam = boto3.client('iam')
sts = boto3.client('sts')


# First validate that the script won't overwrite an existing role or policy. If you receive
# either exception, change the global variable above to give it a different name
#
# Validate that the script won't overwrite an existing policy.  
existing_policy = None
try:
  account_id = sts.get_caller_identity()['Account']
  policy_arn = f'arn:aws:iam::{account_id}:policy/{create_connector_policy_name}'
  existing_policy = iam.get_policy(PolicyArn=policy_arn)['Policy']
  if existing_policy:
    raise Exception(f"Policy {create_connector_policy_name} already exists. Please set another policy name")
except iam.exceptions.NoSuchEntityException:
  # The policy document does not exist. That's the expected result, so there's
  # nothing additional to do
  pass


# Validate that the script won't overwrite an existing role.
existing_role = None
try:
  existing_role = iam.get_role(RoleName=create_connector_role_name)
  if existing_role:
    raise Exception(f"Role {create_connector_role_name} already exists. Please set another role name")
except iam.exceptions.NoSuchEntityException:
  # The role does not exist. That's the expected result, so there's
  # nothing additional to do
  pass


# Create the policy and role. Note, in actual usage, you should wrap these calls
# in try/except blocks and validate the responses. 
#
# Create the policy
policy = iam.create_policy(
  PolicyName=create_connector_policy_name,
  PolicyDocument=json.dumps(policy)
)
policy_arn = policy['Policy']['Arn']


# Create the role
role = iam.create_role(
  RoleName=create_connector_role_name,
  AssumeRolePolicyDocument=json.dumps(trust_relationship)
)
role_arn = role['Role']['Arn']
iam.attach_role_policy(
  RoleName=create_connector_role_name,
  PolicyArn=policy_arn
)

print(f'Created policy {policy_arn}')
print(f'Created role {role_arn}')
# print(f'\nPlease execute the following command\nos.environ["CREATE_DEEPSEEK_CONNECTOR_ROLE"] = "{role_arn}"\n')

os.environ["CREATE_DEEPSEEK_CONNECTOR_ROLE"] = role_arn

Created policy arn:aws:iam::812046859005:policy/create_deepseek_connector_policy
Created role arn:aws:iam::812046859005:role/create_deepseek_connector_role


## 2.3 配置 OpenSearch 中的安全策略

将 create_deepseek_connector_role 添加到 OpenSearch 的安全策略中

In [19]:
import boto3
from opensearchpy import OpenSearch
import os


# Read the configuration from environment variables.
opensearch_service_api_endpoint = 'https://search-public-vector-4xl-7tpwyy5762g6qmi3xukiasfmhq.us-east-1.es.amazonaws.com'
opensearch_user_name = 'admin'
opensearch_user_password = 'Amazon123!'
create_deepseek_connector_role = 'Created role arn:aws:iam::812046859005:role/create_deepseek_connector_role'
lambda_invoke_ml_commons_role_name = 'LambdaInvokeOpenSearchMLCommonsRole'
opensearch_port = 443

# Ensure the endpoint matches the contract for the opensearch-py client. Endpoints
# are specified without the leading URL scheme or trailing slashes.
if opensearch_service_api_endpoint.startswith('https://'):
  opensearch_service_api_endpoint = opensearch_service_api_endpoint[len('https://'):]
if opensearch_service_api_endpoint.endswith('/'):
  opensearch_service_api_endpoint = opensearch_service_api_endpoint[:-1]


# Construct the backend roles. OpenSearch's fine-grained access control will detect
# signed traffic and map these entities to the ml_full_access role.
sts = boto3.client('sts')
account_id = sts.get_caller_identity()['Account']
lambda_invoke_ml_commons_role_arn = f'arn:aws:iam::{account_id}:role/{lambda_invoke_ml_commons_role_name}'
role_mapping = {
  "backend_roles": [create_deepseek_connector_role,
                    lambda_invoke_ml_commons_role_arn]
}

hosts = [{"host": opensearch_service_api_endpoint, "port": opensearch_port}]
client = OpenSearch(
    hosts=hosts,
    http_auth=(opensearch_user_name, opensearch_user_password),
    use_ssl=True,
    verify_certs=False,
    ssl_assert_hostname=False,
    ssl_show_warn=False,
)
# client.security.create_role_mapping('ml_full_access', body=role_mapping)

# 使用 transport.perform_request 方法直接调用 API
client.transport.perform_request(
    'PUT',
    '/_plugins/_security/api/rolesmapping/ml_full_access',
    body=role_mapping
)

# 获取角色映射
response = client.transport.perform_request(
    'GET',
    '/_plugins/_security/api/rolesmapping/ml_full_access'
)
print(f'ml_full_access role mapping is now {response}')


ml_full_access role mapping is now {'ml_full_access': {'hosts': [], 'users': [], 'reserved': False, 'hidden': False, 'backend_roles': ['Created role arn:aws:iam::812046859005:role/create_deepseek_connector_role', 'arn:aws-cn:iam::812046859005:role/LambdaInvokeOpenSearchMLCommonsRole'], 'and_backend_roles': []}}


# 3. 创建 ML Connector

## 3.1 创建 DeepSeek Model Connector

In [6]:
import boto3
import requests 
from requests_aws4auth import AWS4Auth
import json
import os

host = 'https://search-public-vector-4xl-7tpwyy5762g6qmi3xukiasfmhq.us-east-1.es.amazonaws.com'
region = 'us-east-1'
service = 'es'

credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service, session_token=credentials.token)

path = '/_plugins/_ml/connectors/_create'
url = host + path

payload = {
  "name": "DeepSeek R1 model connector",
  "description": "Connector for my Bedrock DeepSeek model",
  "version": "1.0",
  "protocol": "aws_sigv4",
  "credential": {
    "roleArn": "arn:aws:iam::812046859005:role/my_invoke_bedrock_deepseek_model_role"
  },
  "parameters": {
    "service_name": "bedrock",
    "region": "us-east-1",
    "model_id": "us.deepseek.r1-v1:0",
    "temperature": 0.5,
    "max_tokens": 4000
  },
  "actions": [
    {
      "action_type": "PREDICT",
      "method": "POST",
      # "url": "https://bedrock-runtime.${parameters.region}.amazonaws.com/model/inference-profile/${parameters.model_id}/invoke",
      "url": "https://bedrock-runtime.${parameters.region}.amazonaws.com/model/${parameters.model_id}/invoke",
      "headers": {
        "content-type": "application/json"
      },
      "request_body": "{ \"prompt\": \"<｜begin▁of▁sentence｜><｜User｜>${parameters.inputs}<｜Assistant｜>\", \"temperature\": ${parameters.temperature}, \"max_tokens\": ${parameters.max_tokens} }",
      # "post_process_function": "\n      return '{' +\n               '\"name\": \"response\",'+\n               '\"dataAsMap\": {' +\n                  '\"completion\":\"' + params.choices[0].text + '\"}' +\n             '}';\n    "
       "post_process_function": """
            try {
            // 从响应中提取生成的文本
            def completion = "";
            
            // 检查是否存在 choices 数组
            if (params.choices.size() > 0) {
              // 获取第一个选择的文本
              completion = params.choices[0].text;
              
            } else {
              completion = "无法从模型响应中提取文本";
            }
            
            
            def json = "{" +
                             "\\"name\\": \\"response\\","+
                            "\\"dataAsMap\\": {" +
                             "\\"completion\\":\\"" + escape(completion) + "\\"}" +
                             "}";
            return json;
            
        
          } catch (Exception e) {
            // 返回错误信息
            return '{' +
                     '"name": "response",' +
                     '"dataAsMap": {' +
                        '"completion": "处理错误: ' + escape(e) + '"' +
                     '}' +
                   '}';
          }
        """
    }
  ]
}

headers = {"Content-Type": "application/json"}

r = requests.post(url, auth=awsauth, json=payload, headers=headers)
print(r.status_code)
print(r.text)
connector_id = json.loads(r.text)['connector_id']
os.environ["DEEPSEEK_CONNECTOR_ID"] = connector_id

200
{"connector_id":"RvOgIZcBnlXe1e0nSop7"}


## 3.2 注册 DeepSeek Model

In [7]:
import os
import requests


opensearch_service_api_endpoint = 'https://search-public-vector-4xl-7tpwyy5762g6qmi3xukiasfmhq.us-east-1.es.amazonaws.com'
opensearch_user_name = 'admin'
opensearch_user_password = 'Amazon123!'
region = 'us-east-1'
connector_id = os.environ["DEEPSEEK_CONNECTOR_ID"]


# Set up user/password auth
userauth = (opensearch_user_name, opensearch_user_password)
headers = {"Content-Type": "application/json"}


########################################################################################
# Register the model
path = '/_plugins/_ml/models/_register'
url = opensearch_service_api_endpoint + path
payload = {
  "name": "Bedrock DeepSeek R1 model",
  "function_name": "remote",
  "description": "DeepSeek R1 model on Bedrock",
  "connector_id": connector_id
}
r = requests.post(url, auth=userauth, json=payload, headers=headers)

model_id = r.json()['model_id']
print(f'model_id: {model_id}')


########################################################################################
# Deploy the model
path = f'/_plugins/_ml/models/{model_id}/_deploy'
url = opensearch_service_api_endpoint + path
r = requests.post(url, auth=userauth, headers=headers)


print(f'DEEPSEEK_MODEL_ID="{model_id}"\n')
os.environ["DEEPSEEK_MODEL_ID"] = model_id

model_id: SPOgIZcBnlXe1e0nU4po
DEEPSEEK_MODEL_ID="SPOgIZcBnlXe1e0nU4po"



## 4.3 测试模型

在 OpenSearch Dashboard 中的 Dev Tool 中执行如下命令 <p>
```shell
POST _plugins/_ml/models/<modelid>/_predict
{  
   "parameters": {    
      "inputs": "OpenSearch Serverless 是什么，和OpenSearch集群模式有什么区别，使用 OpenSearch Serverless，还需要管理服务器资源么？"
   }
}
```

In [ ]:
# Copyright 2025 Norris
# MIT-0
#

import os
import requests
from pprint import pprint


opensearch_service_api_endpoint = os.environ['OPENSEARCH_SERVICE_DOMAIN_ENDPOINT']
opensearch_user_name = os.environ['OPENSEARCH_SERVICE_ADMIN_USER']
opensearch_user_password = os.environ['OPENSEARCH_SERVICE_ADMIN_PASSWORD']
region = os.environ['DEEPSEEK_AWS_REGION']
connector_id = os.environ['EMBEDDING_CONNECTOR_ID']
create_deepseek_connector_role = os.environ['CREATE_DEEPSEEK_CONNECTOR_ROLE']
deepseek_model_id = os.environ["DEEPSEEK_MODEL_ID"]

# Set up user/password auth
userauth = (opensearch_user_name, opensearch_user_password)
headers = {"Content-Type": "application/json"}


########################################################################################
# Test LLM model
path = f'/_plugins/_ml/models/{deepseek_model_id}/_predict'
url = opensearch_service_api_endpoint + path
payload = {
  "parameters": {    
      "inputs": "OpenSearch Serverless 是什么，和OpenSearch集群模式有什么区别，使用 OpenSearch Serverless，还需要管理服务器资源么？"
   }
}
r = requests.post(url, auth=userauth, json=payload, headers=headers)

response_content = r.json()['inference_results'][0]['output'][0]['dataAsMap']
pprint(f'response: {response_content}')

# 6.创建 Search Pipeline

**Retrieval-augmented generation (RAG) processor**

RAG processor 是一个搜索结果处理器，主要用于对话式搜索中的检索增强生成<br>
* 将向量引擎的检索能力与 LLM 的生成能力无缝结合，无需手动编写复杂管道代码，只需配置即可实现“检索-增强生成”的端到端流程。
* 隐藏底层向量搜索、提示词拼接、模型调用等细节，将提示（prompt）发送给大语言模型（LLM）
* 自动将 OpenSearch 检索到的相关文档作为上下文注入 LLM 提示词，提升生成结果的准确性和相关性，避免幻觉问题。

在 OpenSearch Dashboard 中的 Dev Tool 中执行如下命令，创建混合查询的 search pileline
```
PUT /_search/pipeline/my-conversation-search-pipeline-deepseek-zh
{
    "response_processors": [
      {
        "retrieval_augmented_generation": {
          "tag": "Demo pipeline",
          "description": "Demo pipeline Using DeepSeek R1",
          "model_id": "SPOgIZcBnlXe1e0nU4po",
          "context_field_list": [
            "text"
          ],
          "system_prompt": "你是一个Amazon EMR版本升级助手.",
          "user_instructions": "针对给定的问题，用少于 200 个字给出简洁、翔实的答案"
        }
      }
    ],
    "phase_results_processors": [
      {
        "normalization-processor": {
          "normalization": {
            "technique": "min_max"
          },
          "combination": {
            "technique": "arithmetic_mean",
            "parameters": {
              "weights": [
                0.6,
                0.4
              ]
            }
          }
        }
      }
    ]
}
```

In [ ]:
# Copyright 2025 Norris
# MIT-0
#

import os
import requests


opensearch_service_api_endpoint = os.environ['OPENSEARCH_SERVICE_DOMAIN_ENDPOINT']
opensearch_user_name = os.environ['OPENSEARCH_SERVICE_ADMIN_USER']
opensearch_user_password = os.environ['OPENSEARCH_SERVICE_ADMIN_PASSWORD']
region = os.environ['DEEPSEEK_AWS_REGION']
connector_id = os.environ['EMBEDDING_CONNECTOR_ID']
create_deepseek_connector_role = os.environ['CREATE_DEEPSEEK_CONNECTOR_ROLE']
deepseek_model_id = os.environ["DEEPSEEK_MODEL_ID"]

# Set up user/password auth
userauth = (opensearch_user_name, opensearch_user_password)
headers = {"Content-Type": "application/json"}


########################################################################################
# Create Search Pipeline
path = f'/_search/pipeline/my-conversation-search-pipeline-deepseek-zh'
url = opensearch_service_api_endpoint + path
payload = {
    "response_processors": [
        {
            "retrieval_augmented_generation": {
                "tag": "Demo pipeline",
                "description": "Demo pipeline Using DeepSeek R1",
                "model_id": deepseek_model_id,
                "context_field_list": [
                    "text"
                ],
                "system_prompt": "你是一个Amazon EMR版本升级助手.",
                "user_instructions": "针对给定的问题，用少于 200 个字给出简洁、翔实的答案"
            }
        }
    ],
    "phase_results_processors": [
        {
            "normalization-processor": {
                "normalization": {
                    "technique": "min_max"
                },
                "combination": {
                    "technique": "arithmetic_mean",
                    "parameters": {
                        "weights": [
                            0.6,
                            0.4
                        ]
                    }
                }
            }
        }
    ]
}
r = requests.put(url, auth=userauth, json=payload, headers=headers)

reponse = r.json()
print(f'reponse: {reponse}')

# 7.测试问答

在 OpenSearch Dashboard 中的 Dev Tool 中执行如下命令
```
GET opensearch_kl_index/_search?search_pipeline=my-conversation-search-pipeline-deepseek-zh
{
  "query": {
    "neural": {
    "text_embedding": {
        "query_text": "OpenSearch Serverless 是什么，和OpenSearch集群模式有什么区别，使用 OpenSearch Serverless，还需要管理服务器资源么？",
        "model_id": "<embedding-model-id>",
        "k": 5
      }
    }
  },
  "size": 2,
  "_source": [
    "text"
  ],
  "ext": {
    "generative_qa_parameters": {
      "llm_model": "bedrock/claude",
      "llm_question": "OpenSearch Serverless 是什么，和OpenSearch集群模式有什么区别，使用 OpenSearch Serverless，还需要管理服务器资源么？",
      "context_size": 5,
      "timeout": 15
    }
  }
}
```

In [14]:
question = "我要升级 Amazon EMR, hive 2.4 升级到 3.1.4 你能帮我检查之前的 SQL 能否在 3.1.4 版本上运行么？"

In [17]:
# Copyright 2025 Norris
# MIT-0
#

# Test

import os
import requests
from pprint import pprint


opensearch_service_api_endpoint = os.environ['OPENSEARCH_SERVICE_DOMAIN_ENDPOINT']
opensearch_user_name = os.environ['OPENSEARCH_SERVICE_ADMIN_USER']
opensearch_user_password = os.environ['OPENSEARCH_SERVICE_ADMIN_PASSWORD']
embedding_mode_id = "TfO5IZcBnlXe1e0nSYoi"

# Set up user/password auth
userauth = (opensearch_user_name, opensearch_user_password)
headers = {"Content-Type": "application/json"}



########################################################################################
# Create Search Pipeline
path = f'/emr_upgrade_hive/_search?search_pipeline=my-conversation-search-pipeline-deepseek-zh'
url = opensearch_service_api_endpoint + path
payload = {
  "query": {
    "neural": {
    "embedding": {
        "query_text": question,
        "model_id": embedding_mode_id,
        "k": 5
      }
    }
  },
  "size": 2,
  "_source": [
    "text"
  ],
  "ext": {
    "generative_qa_parameters": {
      "llm_model": "bedrock/claude",
      "llm_question": question,
      "context_size": 5,
      "timeout": 15
    }
  }
}
response = requests.post(url, auth=userauth, json=payload, headers=headers)

pprint(response.json())

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'ext': {'retrieval_augmented_generation': {'answer': '<think>\n'
                                                      '好的，用户想从Hive '
                                                      '2.4升级到3.1.4，需要检查现有SQL是否兼容。首先，我需要回顾Hive '
                                                      '3.x的变化，特别是可能影响SQL执行的部分。\n'
                                                      '\n'
                                                      '看用户提供的两个搜索结果，里面有很多Hive的JIRA问题。比如HIVE-17672升级了Calcite到1.14，这可能影响查询优化器，导致执行计划不同，但SQL语法可能不会有问题。HIVE-17857升级了ORC到1.4，需要注意ORC表的兼容性，但通常向后兼容。HIVE-18436升级到Spark '
                                                      '2.3.0，如果用户使用Hive on '
                                                      'Spark，可能会有影响，但SQL本身可能没问题。\n'
                                                      '\n'
                                                      '另外，HIVE-18598提到外部表不允许NOT '
                                              

<br>
<br>
<br>
<br>

# 8.对话式搜索(创建对话记忆)

对话式搜索允许您用自然语言提问，并通过提出后续问题来完善答案。因此，对话变成了您与大型语言模型 (LLM) 之间的对话。<br>为此，模型需要记住整个对话的上下文，而不是单独回答每个问题。
在 OpenSearch Dashboard 中的 Dev Tool 中执行如下命令
```
POST /_plugins/_ml/memory/
{
  "name": "Conversation about DeepSeek Demo"
}
```

In [ ]:
# Copyright 2025 Norris
# MIT-0
#

import os
import requests


opensearch_service_api_endpoint = os.environ['OPENSEARCH_SERVICE_DOMAIN_ENDPOINT']
opensearch_user_name = os.environ['OPENSEARCH_SERVICE_ADMIN_USER']
opensearch_user_password = os.environ['OPENSEARCH_SERVICE_ADMIN_PASSWORD']
region = os.environ['DEEPSEEK_AWS_REGION']
connector_id = os.environ['EMBEDDING_CONNECTOR_ID']
create_deepseek_connector_role = os.environ['CREATE_DEEPSEEK_CONNECTOR_ROLE']
deepseek_model_id = os.environ["DEEPSEEK_MODEL_ID"]

# Set up user/password auth
userauth = (opensearch_user_name, opensearch_user_password)
headers = {"Content-Type": "application/json"}


########################################################################################
# Test LLM model
path = f'/_plugins/_ml/memory/'
url = opensearch_service_api_endpoint + path
payload = {
  "name": "Conversation about DeepSeek Demo"
}
r = requests.post(url, auth=userauth, json=payload, headers=headers)

memory_id = r.json()['memory_id']
print(f'memory_id: {memory_id}')
os.environ["MEMORY_ID"] = memory_id

### 带有记忆功能的搜索

In [ ]:
question = "OpenSearch Serverless 是什么，和OpenSearch集群模式有什么区别，使用 OpenSearch Serverless，还需要管理服务器资源么？"

In [ ]:
question = "能否再详细解释一下"

In [ ]:
# Copyright 2025 Norris
# MIT-0
#

# Test

import os
import requests
from pprint import pprint


opensearch_service_api_endpoint = os.environ['OPENSEARCH_SERVICE_DOMAIN_ENDPOINT']
opensearch_user_name = os.environ['OPENSEARCH_SERVICE_ADMIN_USER']
opensearch_user_password = os.environ['OPENSEARCH_SERVICE_ADMIN_PASSWORD']
region = os.environ['DEEPSEEK_AWS_REGION']
connector_id = os.environ['EMBEDDING_CONNECTOR_ID']
create_deepseek_connector_role = os.environ['CREATE_DEEPSEEK_CONNECTOR_ROLE']
embedding_mode_id = os.environ["EMBEDDING_MODEL_ID"]
memory_id = os.environ["MEMORY_ID"]

# Set up user/password auth
userauth = (opensearch_user_name, opensearch_user_password)
headers = {"Content-Type": "application/json"}



########################################################################################
# Create Search Pipeline
path = f'/opensearch_kl_index/_search?search_pipeline=my-conversation-search-pipeline-deepseek-zh'
url = opensearch_service_api_endpoint + path
payload = {
  "query": {
    "neural": {
    "text_embedding": {
        "query_text": question,
        "model_id": embedding_mode_id,
        "k": 5
      }
    }
  },
  "size": 2,
  "_source": [
    "text"
  ],
  "ext": {
    "generative_qa_parameters": {
      "llm_model": "bedrock/claude",
      "llm_question": question,
      "memory_id": memory_id,
      "context_size": 2,
      "message_size": 2,
      "timeout": 15
    }
  }
}
response = requests.post(url, auth=userauth, json=payload, headers=headers)

pprint(response.json())

#### 查看记忆中的消息
```
GET /_plugins/_ml/memory/<memory-id>/messages
```

In [48]:
import boto3
import json

def invoke_deepseek_r1(prompt):
    # 创建 Bedrock Runtime 客户端
    bedrock_runtime = boto3.client(
        service_name='bedrock-runtime',
        region_name='us-east-1'  # 根据你的区域调整
    )
    
    # DeepSeek R1 模型 ID
    model_id = "us.deepseek.r1-v1:0"  # 确认正确的模型 ID
    
    # 构建请求体
    request_body = {
        "prompt": prompt,
        "max_tokens": 1000,
        "temperature": 0.7,
        "top_p": 0.9
    }
    
    try:
        # 调用模型
        response = bedrock_runtime.invoke_model(
            modelId=model_id,
            body=json.dumps(request_body)
        )
        
        # 解析响应
        response_body = json.loads(response.get('body').read())
        return response_body
        
    except Exception as e:
        print(f"调用模型时出错: {str(e)}")
        return None

# 使用示例
if __name__ == "__main__":
    prompt = "请解释量子计算的基本原理"
    result = invoke_deepseek_r1(prompt)
    
    if result:
        # 根据 DeepSeek R1 的响应格式提取生成的文本
        # 注意：具体的响应格式可能需要根据实际API调整
        if "completion" in result:
            print(result["completion"])
        else:
            print("完整响应:", result)


KeyboardInterrupt: 